In [183]:
# Importing the important libraries
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout,Convolution2D,MaxPooling2D,Flatten,BatchNormalization,Activation,Conv2D
from keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler,TensorBoard,ReduceLROnPlateau


# Utility
import os
import numpy as np
import itertools
import random
from collections import Counter
from glob import iglob

In [184]:
# https://github.com/pmarcelino/blog/blob/master/dogs_cats/dogs_cats.ipynb
# https://cv-tricks.com/keras/understand-implement-resnets/
# https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/

# Data Augmentation
# https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/

# https://yashuseth.blog/2018/11/26/hyper-parameter-tuning-best-practices-learning-rate-batch-size-momentum-weight-decay/
# https://machinelearningmastery.com/understand-the-dynamics-of-learning-rate-on-deep-learning-neural-networks/

# https://chrisalbon.com/machine_learning/support_vector_machines/svc_parameters_using_rbf_kernel/

In [185]:
IMAGE_SIZE = (256,256)
INPUT_SHAPE = (256,256,3)

BATCH_SIZE = 12

In [186]:
base_dir = 'Recycling_Data_V3'
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
# validation_dir = os.path.join(base_dir, 'validation')
# predict_dir = os.path.join(base_dir, 'predict')

In [187]:
# Creating objects for image augmentations
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range=90,
                                   vertical_flip=True,
                                   horizontal_flip = True)
 
test_datagen = ImageDataGenerator(rescale = 1./255)

datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
generator = datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=IMAGE_SIZE,  # All images will be resized to 150x150
        batch_size=BATCH_SIZE,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=IMAGE_SIZE,  # All images will be resized to 150x150
        batch_size=BATCH_SIZE,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')


# Flow test images in batches of 20 using train_datagen generator
test_generator = test_datagen.flow_from_directory(
        directory=test_dir,  # This is the source directory for training images
        target_size=IMAGE_SIZE,  # All images will be resized to 150x150
        batch_size=BATCH_SIZE,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='categorical')

Found 1588 images belonging to 4 classes.
Found 1588 images belonging to 4 classes.
Found 317 images belonging to 4 classes.


In [188]:
# Download the architecture of ResNet50 with ImageNet weights
base_model = ResNet50(include_top=False, weights='imagenet',input_shape=INPUT_SHAPE)

C:\Users\kaush\anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [189]:
x = base_model.output

In [190]:
# Adding a Global Average Pooling layer
x = GlobalAveragePooling2D()(x)

In [191]:
# x = BatchNormalization()(x) 

In [192]:
x = Dropout(0.5)(x)  
x = Dense(1024, activation ='relu')(x) 
x = BatchNormalization()(x) 
x = Dropout(0.5)(x) 

In [193]:

# x = Dense(1024, activation='relu')(x)

In [194]:
# Adding a fully connected layer having 2 neurons which will
# give the probability of image having either dog or cat
predictions = Dense(4, activation='softmax')(x)
 
# Model to be trained
model = Model(inputs=base_model.input, outputs=predictions) 

In [195]:
# Compiling the model
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9, decay=0.0001), loss='categorical_crossentropy', metrics = ['accuracy'])

earlystopping = EarlyStopping(monitor='loss',patience=5,verbose=1)


# Training only top layers i.e. the layers which we have added in the end
# for layer in base_model.layers:
#     layer.trainable = False

In [196]:

history=model.fit_generator(train_generator,
                    steps_per_epoch = 50,
                    epochs=12,
                    validation_data=test_generator,validation_steps=10,
                    callbacks=[earlystopping])

Epoch 1/12
50/50 [==============================] - 394s 8s/step - loss: 2.1758 - accuracy: 0.3067 - val_loss: 1.6367 - val_accuracy: 0.1917
Epoch 2/12
50/50 [==============================] - 396s 8s/step - loss: 1.7103 - accuracy: 0.4409 - val_loss: 1.5621 - val_accuracy: 0.2833
Epoch 3/12
50/50 [==============================] - 455s 9s/step - loss: 1.3002 - accuracy: 0.5483 - val_loss: 2.4023 - val_accuracy: 0.3009
Epoch 4/12
50/50 [==============================] - 455s 9s/step - loss: 1.1852 - accuracy: 0.5817 - val_loss: 2.8473 - val_accuracy: 0.1917
Epoch 5/12
50/50 [==============================] - 379s 8s/step - loss: 1.0638 - accuracy: 0.6368 - val_loss: 1.3088 - val_accuracy: 0.2667
Epoch 6/12
50/50 [==============================] - 427s 9s/step - loss: 1.0330 - accuracy: 0.6433 - val_loss: 3.3129 - val_accuracy: 0.1770
Epoch 7/12
50/50 [==============================] - 421s 8s/step - loss: 0.9000 - accuracy: 0.6858 - val_loss: 2.8641 - val_accuracy: 0.2583
Epoch 8/12
50

In [197]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

res5a_branch2a (Conv2D)         (None, 8, 8, 512)    524800      activation_334[0][0]             
__________________________________________________________________________________________________
bn5a_branch2a (BatchNormalizati (None, 8, 8, 512)    2048        res5a_branch2a[0][0]             
__________________________________________________________________________________________________
activation_335 (Activation)     (None, 8, 8, 512)    0           bn5a_branch2a[0][0]              
__________________________________________________________________________________________________
res5a_branch2b (Conv2D)         (None, 8, 8, 512)    2359808     activation_335[0][0]             
__________________________________________________________________________________________________
bn5a_branch2b (BatchNormalizati (None, 8, 8, 512)    2048        res5a_branch2b[0][0]             
__________________________________________________________________________________________________
activation

In [198]:
from sklearn.svm import SVC

In [199]:
param = [{
          "C": [1000]
         }]

In [200]:
# svm_features = np.concatenate((train_generator, test_generator))

In [201]:
train_generator.class_indices

{'glass': 0, 'metal': 1, 'paper': 2, 'plastic': 3}

In [202]:
# svm.fit(train_generator,np.argmax(test_generator,axis=1))

In [203]:
# model_feat = Model(inputs=model.input,outputs=model.get_layer('dense_9').output)

# feat_train = model_feat.predict(train_generator)

In [204]:
# feat_train

In [205]:
# datagen = ImageDataGenerator(rescale=1./255)
# batch_size = 12

# def extract_features(directory, sample_count): 
#     labels = np.zeros(shape=(sample_count,4))
# #     print(labels.shape) 
#     i = 0
#     for inputs_batch, labels_batch in generator:
# #         print(labels_batch.shape)
#         labels[i * batch_size: (i + 1) * batch_size] = labels_batch
#         i += 1
# #         print(i)
#         if i * batch_size >= sample_count:
#             break
#     return labels
    
# train_labels = extract_features(train_dir, 1588)

In [206]:
# train_labels.shape
# feat_train


In [207]:
# svm = SVC(kernel='rbf',C=1000,gamma=0.5)

# # svm = SVC(kernel='linear', C=1)
# svm.fit(feat_train,np.argmax(train_labels,axis=1))

# print('fitting done !!!')

In [208]:
# svm.score(feat_train,np.argmax(train_labels,axis=1))

In [209]:
# model.save_weights('bottleneck_fc_model.h5')

In [210]:
result = model.evaluate(test_generator) 
  
print('The model achieved a loss of %.2f and,'
      'accuracy of %.2f%%.' % (result[0], result[1]*100)) 

27/27 [==============================] - 91s 3s/step
The model achieved a loss of 4.84 and,accuracy of 32.81%.


In [211]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 12

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count,4))  # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,4))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=IMAGE_SIZE,
                                            batch_size = batch_size,
                                            class_mode='categorical')
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 1588)

Found 1588 images belonging to 4 classes.


In [212]:
# train_features.shape
# train_labels

In [213]:
svm = SVC(kernel='rbf',C=1000,gamma=0.5)

# svm = SVC(kernel='linear', C=1)
svm.fit(train_features,np.argmax(train_labels,axis=1))

print('fitting done !!!')

fitting done !!!


In [214]:
svm.score(train_features,np.argmax(train_labels,axis=1))

0.45025188916876574

In [215]:
# 0.4005037783375315
# 0.45025188916876574